# 01 Basic Chat model & Chain - Langchain

## 1. Prerequisites

- **Install Python** : Ensure that Python is installed on your system. You can download it from the official website: [Python Downloads](https://www.python.org/downloads/).

- **Install Dependencies** : After installing Python, use `pip` to install the required dependencies:
     ```bash
     pip install -r requirements.txt
     ```

## 2. Chat Models

- In this course, we'll be utilizing [Chat Models](https://python.langchain.com/v0.2/docs/concepts/#chat-models), which process a series of messages as inputs and generate chat responses as outputs. LangChain itself doesn't host any Chat Models, so we rely on integrations with third-party services.
- For this course, we'll primarily use [AzureChatOpenAI](https://python.langchain.com/v0.2/docs/integrations/chat/azure_chat_openai/) due to its popularity and performance. Please make sure you have the `OPENAI_API_VERSION`, `AZURE_OPENAI_ENDPOINT`, and `AZURE_OPENAI_API_KEY` set correctly.
- Let's verify if your `OPENAI_API_VERSION`, `AZURE_OPENAI_ENDPOINT`, and `AZURE_OPENAI_API_KEY` are included in the `.env` file. If they're missing, you'll be prompted to provide them.

There are [a few standard parameters](https://python.langchain.com/v0.2/docs/concepts/#chat-models) for chat models, including:

* `model`: the model name
* `temperature`: controls randomness
* `azure_deployment`: the name of your Azure deployment
* `api_version`: specifies the API version
* `api_key`: the API key for authentication
* `azure_endpoint`: the Azure endpoint for the service

A low `temperature` (close to 0) produces more deterministic, accurate responses, while a high `temperature` (close to 1) generates more creative and varied outputs. The other parameters help configure the model's deployment and access.

In [9]:
import os
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI

# Step 1: Load environment variables from .env file
load_dotenv(".env")

# Step 2: Retrieve Azure OpenAI environment variables
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
OPENAI_API_VERSION = os.getenv("OPENAI_API_VERSION")

model = AzureChatOpenAI(
    azure_deployment= "gpt-4o-mini",  # Your Azure deployment
    api_version     = OPENAI_API_VERSION,  # Your API version
    api_key         = AZURE_OPENAI_API_KEY,
    azure_endpoint  = AZURE_OPENAI_ENDPOINT,
    temperature     = 0,
    max_tokens      = None,
    timeout         = None,
)

In [13]:
!pip list

Package                                           Version
------------------------------------------------- ---------------
absl-py                                           2.0.0
accelerate                                        0.28.0
ai-agents                                         0.0.52
aiodns                                            3.1.1
aiofiles                                          23.2.1
aiohttp                                           3.8.6
aiohttp-cors                                      0.7.0
aiohttp-retry                                     2.8.3
aiohttp-socks                                     0.8.4
aiolimiter                                        1.1.0
aiorwlock                                         1.3.0
aiosignal                                         1.3.1
alabaster                                         0.7.16
alembic                                           1.13.1
altair                                            5.1.2
amazon-textract-caller         

In [27]:
from langchain_community.callbacks.manager import get_openai_callback
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

# Create a prompt template for insurance-related query
prompt = PromptTemplate(
    template="Tell me a quick fact about insurance regarding {topic}",
    input_variables=['topic']
)

# Output parser for the response
output_parser = StrOutputParser()

# Chain setup with model and output parser
chain = prompt | model | output_parser
with get_openai_callback() as cb:
# Invoke with insurance-related topic
    result = chain.invoke({"topic": "health insurance in Thailand"})


In [28]:
print(cb)

Tokens Used: 69
	Prompt Tokens: 19
	Completion Tokens: 50
Successful Requests: 1
Total Cost (USD): $3.285e-05


In [29]:
cb_dict = cb.__dict__.pop('_lock', None)

In [31]:
cb.__dict__

{'total_cost': 3.285e-05,
 'total_tokens': 69,
 'prompt_tokens': 19,
 'completion_tokens': 50,
 'successful_requests': 1}

In [129]:
from typing import Literal, Optional
from pydantic import BaseModel, Field

class UserResponse(BaseModel):
    reasoning : str = Field(description="Explain in english why intent was chosen during conversation"),
    definition : Literal[
        '0. General: Questions or statements that do not fall into specific categories or lack clear intent, such as greetings, casual inquiries, or non-relevant statements',
        '1. Information: Requests for general details, product searches, contact department, or clarification about insurance policies, processes, or terms e.g., What is included in a comprehensive car insurance plan',
        '2. Recommendation: Queries seeking advice or suggestions for selecting insurance products, services, or coverage options. e.g., What plan would you recommend for a new driver?',
        '3. Current Status: Inquiries specifically asking to check or view details of an existing active insurance policy. e.g., Can you show me my current policy?, What coverage is included in my policy?, or Is my insurance policy still active?',
        '4. Payment: Questions related to premium payments, due dates, methods of payment, or any financial transactions e.g., How can I pay my premium online?',
        '5. Claim: Focuses solely on checking the claim status or submitting a new claim request only e.g., What is the status of my claim?, I need to file a claim for my car accident.',
        '6. Quotation: Purchasing or inquiring about purchasing specific insurance policies. This category is only for when the customer would like to purchase or inquire about purchasing a product e.g., Can you give me a quote for life insurance?',
        '7. Complaint: Statements expressing dissatisfaction or issues with services, policies, or processes. This intent also includes when the customer is asking for contact with employees. e.g., I am not happy with how my claim was handled.',
        '8. Cancel: The customer is requesting to cancel their insurance policy or asking for details about cancellation processes.',
        '9. Renewal Notice: The customer is asking about the renewal of their policy, renewal dates, or related notifications.',
    ]
    text: str = Field(description="Number of the intent")
    label: str = Field(description="Label Name of the intent") 
    # label: Literal[
    #     'General', 
    #     'Information', 
    #     'Recommendation',
    #     'Current Status',
    #     'Payment',
    #     'Claim',
    #     'Quotation',
    #     'Complaint',
    #     'Cancel',
    #     'Renewal Notice'
    #     ]
    # listing_id: Optional[str]
    
CLASSIFIER_GPT = AzureChatOpenAI(
    model="gpt-4o-mini", 
    azure_deployment= "gpt-4o-mini",  # Your Azure deployment
    api_version     = "2024-08-01-preview", #OPENAI_API_VERSION,  # Your API version
    api_key         = AZURE_OPENAI_API_KEY,
    azure_endpoint  = AZURE_OPENAI_ENDPOINT,
    temperature     = 0,
    response_format = UserResponse)

/opt/homebrew/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3488: UserWarning: WARNING! response_format is not default parameter.
                response_format was transferred to model_kwargs.
                Please confirm that response_format is what you intended.
  if await self.run_code(code, result, async_=asy):


In [135]:
# !pip list langchain-openai #0.2.8

In [130]:
message = "human:สวัสดีครับ\nchatbot:สวัสดีค่ะ น้องหงส์ยินดีให้บริการค่ะ มีอะไรให้ช่วยเหลือหรือสอบถามเกี่ยวกับประกันภัยจากไทยไพบูลย์ประกันภัยไหมคะ?\nhuman:ขอดูกรมธรรม์หน่อยครับ"
message = "Human: สวัสดีครับ\nAI: สวัสดีค่ะ ยินดีให้บริการ\nHuman: ผมกำลังมองหาประกันรถยนต์\nAI: มีแผนประกันแบบไหนในใจหรือเปล่าครับ\nHuman: ขอคำแนะนำสำหรับมือใหม่\nAI: สำหรับมือใหม่แนะนำแผนครอบคลุมเพื่อความอุ่นใจ\nHuman:ขอคำแนะนำเกี่ยวกับประกันรถยนต์สำหรับมือใหม่"
message = "Human: สวัสดีค่ะ\nAI: สวัสดีค่ะ มีอะไรให้ช่วยไหม\nHuman: ฉันต้องการยกเลิกกรมธรรม์"
message = "Human: สวัสดีครับ\nAI: สวัสดีค่ะ ยินดีให้บริการ\nHuman: ฉันยื่นเคลมไว้เมื่อสัปดาห์ที่แล้ว\nAI: กรุณาแจ้งหมายเลขเคลมค่ะ\nHuman: หมายเลขคือ CL123456\nAI: การเคลมของคุณกำลังอยู่ระหว่างดำเนินการ\nHuman:สถานะการเคลมของฉันถึงไหนแล้ว"
# message = "hi"

In [131]:
result  = CLASSIFIER_GPT.invoke(message)
import json
json.loads(result.content)

/opt/homebrew/lib/python3.11/site-packages/pydantic/json_schema.py:2191: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='Explain in english why intent was chosen during conversation'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


{'reasoning': 'The user is inquiring about the current status of their claim, which indicates they want to check the progress or details of an existing claim they have submitted.',
 'definition': '5. Claim: Focuses solely on checking the claim status or submitting a new claim request only e.g., What is the status of my claim?, I need to file a claim for my car accident.',
 'text': '5',
 'label': 'Claim'}

In [128]:
result.response_metadata['token_usage']

{'completion_tokens': 104,
 'prompt_tokens': 467,
 'total_tokens': 571,
 'completion_tokens_details': {'accepted_prediction_tokens': 0,
  'audio_tokens': 0,
  'reasoning_tokens': 0,
  'rejected_prediction_tokens': 0},
 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}

Chat models in LangChain have a number of [default methods](https://python.langchain.com/v0.2/docs/concepts/#runnable-interface). For the most part, we'll be using:

* `stream`: stream back chunks of the response
* `invoke`: call the chain on an input

In [6]:
model.invoke("Does insurance important or not?")

AIMessage(content="Yes, insurance is generally considered important for several reasons:\n\n1. **Financial Protection**: Insurance provides a safety net against unexpected events, such as accidents, illnesses, natural disasters, or theft. It helps cover costs that could otherwise lead to significant financial strain.\n\n2. **Risk Management**: By transferring the risk of loss to an insurance company, individuals and businesses can manage their exposure to potential financial setbacks.\n\n3. **Peace of Mind**: Knowing that you have insurance coverage can provide peace of mind, allowing you to focus on other aspects of your life or business without the constant worry of potential financial loss.\n\n4. **Legal Requirements**: Certain types of insurance, such as auto insurance or workers' compensation, are legally required in many places. Having the necessary coverage helps you comply with the law.\n\n5. **Access to Services**: Health insurance, for example, can provide access to necessary

In [10]:
result = model.invoke("What factors should be considered if I want to purchase health insurance?")

In [11]:
print(result.content)

When considering the purchase of health insurance, several important factors should be taken into account to ensure you choose a plan that meets your needs and budget. Here are some key factors to consider:

1. **Coverage Options**:
   - **Types of Plans**: Understand the different types of health insurance plans available (e.g., HMO, PPO, EPO, POS) and how they work.
   - **Essential Health Benefits**: Ensure the plan covers essential health benefits such as hospitalization, outpatient care, preventive services, maternity care, mental health services, and prescription drugs.

2. **Premiums**:
   - **Monthly Premium**: Consider how much you can afford to pay each month for coverage. This is the amount you pay regardless of whether you use medical services.

3. **Deductibles**:
   - **Annual Deductible**: Look at the deductible amount, which is the amount you must pay out-of-pocket before your insurance starts to cover costs. Higher deductibles often mean lower premiums.

4. **Out-of-Po

- Chat models accept List[BaseMessage] ([messages](https://python.langchain.com/v0.2/docs/concepts/#messages)) as inputs, or objects which can be coerced to messages, including str (converted to `HumanMessage`) and PromptValue.
- Messages have a role (that describes who is saying the message) and a content property.

In [12]:
from langchain_core.messages import SystemMessage, HumanMessage

messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(content="Please list insurance company in Thailand?"),
]

# Invoke the model with a list of messages 
result = model.invoke(messages)
result

AIMessage(content='Here are some notable insurance companies operating in Thailand:\n\n1. **Bangkok Insurance Public Company Limited**\n2. **Thai Life Insurance Public Company Limited**\n3. **Muang Thai Life Assurance Public Company Limited**\n4. **Krungthai-AXA Life Insurance Public Company Limited**\n5. **Generali Thailand**\n6. **TQM Insurance Broker Co., Ltd.**\n7. **Siam Commercial Bank (SCB) Life Assurance**\n8. **Dhipaya Insurance Public Company Limited**\n9. **Chubb Samaggi Insurance Public Company Limited**\n10. **Allianz Ayudhya Assurance Public Company Limited**\n\nThese companies offer a range of insurance products, including life, health, property, and casualty insurance. Always check for the latest information and reviews when considering an insurance provider.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 160, 'prompt_tokens': 22, 'total_tokens': 182, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_to

We get an `AIMessage` response. Also, note that we can just invoke a chat model with a string. When a string is passed in as input, it is converted to a `HumanMessage` and then passed to the underlying model.


In [18]:
print(result.content)

Here are some notable insurance companies operating in Thailand:

1. **Bangkok Insurance Public Company Limited**
2. **Thai Life Insurance Public Company Limited**
3. **Muang Thai Life Assurance Public Company Limited**
4. **Krungthai-AXA Life Insurance Public Company Limited**
5. **Generali Thailand**
6. **TQM Insurance Broker Co., Ltd.**
7. **Siam Commercial Bank (SCB) Life Assurance**
8. **Dhipaya Insurance Public Company Limited**
9. **Chubb Samaggi Insurance Public Company Limited**
10. **Allianz Ayudhya Assurance Public Company Limited**

These companies offer a range of insurance products, including life, health, property, and casualty insurance. Always check for the latest information and reviews when considering an insurance provider.


## 3. Express Language
LCEL makes it easy to build complex chains from basic components, and supports out of the box functionality such as streaming, parallelism, and logging.

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

# Create a prompt template for insurance-related query
prompt = PromptTemplate(
    template="Tell me a quick fact about insurance regarding {topic}",
    input_variables=['topic']
)

# Output parser for the response
output_parser = StrOutputParser()

# Chain setup with model and output parser
chain = prompt | model | output_parser

# Invoke with insurance-related topic
result = chain.invoke({"topic": "health insurance in Thailand"})

Notice this line of the code, where we piece together these different components into a single chain using LCEL:
```bash
chain = prompt | model | output_parser
```

### PromptTemplate

Use PromptTemplate to create a template for a string prompt.
It takes in a dictionary of template variables and produces a PromptValue. 
By default, PromptTemplate uses Python's str.format syntax for templating.

In [20]:
prompt_value = prompt.invoke({"topic": "health insurance in Thailand"})
prompt_value
prompt_value.to_string()

'Tell me a quick fact about insurance regarding health insurance in Thailand'

### Model
The PromptValue is then passed to model. 
In this case our model is a ChatModel, meaning it will output a BaseMessage.

In [21]:
message = model.invoke(prompt_value)
message

AIMessage(content="In Thailand, the Universal Coverage Scheme (UCS) provides health insurance to over 48 million citizens, ensuring access to essential medical services. This scheme is part of the country's effort to provide affordable healthcare and reduce out-of-pocket expenses for its population.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 19, 'total_tokens': 69, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_5154047bf2', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'sa

### Output parser
We pass our model output to the output_parser, which is a BaseOutputParser meaning it takes either a string or a BaseMessage as input. The specific StrOutputParser simply converts any input into a string.

In [22]:
output_parser.invoke(message)

"In Thailand, the Universal Coverage Scheme (UCS) provides health insurance to over 48 million citizens, ensuring access to essential medical services. This scheme is part of the country's effort to provide affordable healthcare and reduce out-of-pocket expenses for its population."

## 4. JSON parser
This output parser allows users to specify an arbitrary JSON schema and query LLMs for outputs that conform to that schema.

Keep in mind that large language models are leaky abstractions! You'll have to use an LLM with sufficient capacity to generate well-formed JSON. 

In [16]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

# Define the data structure for an insurance promotion brochure.
class InsuranceBrochure(BaseModel):
    product_name: str = Field(description="name of the insurance product")
    coverage_details: str = Field(description="details about the coverage offered")
    premium_info: str = Field(description="details about the premium costs")
    target_audience: str = Field(description="intended audience for the insurance product")
    benefits: str = Field(description="key benefits of the insurance product")

# Set up a parser and inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=InsuranceBrochure)

prompt = PromptTemplate(
    template="Extract the necessary details for the insurance brochure.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Assume `model` is defined somewhere in your workflow, e.g., GPT-3 model
chain = prompt | model | parser


# And a query intended to prompt a language model to populate the data structure.
brochure_query = """
Our new HealthGuard Insurance is designed to offer comprehensive protection for you and your family. 
The product provides wide coverage including hospital stays, surgeries, doctor visits, and emergency services. 
It also covers preventive health care like vaccinations and health check-ups. 
The premium starts at just $200 per year for an individual, with discounts for family plans. 
The target audience for this plan is anyone seeking affordable and reliable health insurance with excellent coverage. 
Key benefits include easy claim process, extensive network of healthcare providers, and 24/7 customer support.
"""

brochure_info = chain.invoke({"query": brochure_query})

# Display the result
print(brochure_info)

/opt/homebrew/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3548: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


{'product_name': 'HealthGuard Insurance', 'coverage_details': 'Comprehensive protection including hospital stays, surgeries, doctor visits, emergency services, preventive health care like vaccinations and health check-ups.', 'premium_info': 'Premium starts at just $200 per year for an individual, with discounts for family plans.', 'target_audience': 'Anyone seeking affordable and reliable health insurance with excellent coverage.', 'benefits': 'Easy claim process, extensive network of healthcare providers, and 24/7 customer support.'}


In [18]:
brochure_info

{'product_name': 'HealthGuard Insurance',
 'coverage_details': 'Comprehensive protection including hospital stays, surgeries, doctor visits, emergency services, preventive health care like vaccinations and health check-ups.',
 'premium_info': 'Premium starts at just $200 per year for an individual, with discounts for family plans.',
 'target_audience': 'Anyone seeking affordable and reliable health insurance with excellent coverage.',
 'benefits': 'Easy claim process, extensive network of healthcare providers, and 24/7 customer support.'}

## Workshop: Structuring Personal Medical Information (10mins)

### **Scenario**: **Verifying health customer**
You are building a system to handle medical records where personal medical information needs to be extracted from unstructured text documents. This system will allow healthcare providers to easily categorize and process patient data based on their needs. Your task is to define a structured format for storing key medical information extracted from a patient's medical record.

### **Given Query**:
Here’s an example of a patient's medical document text:

```bash
Patient Name: John Doe
Date of Birth: 1985-06-15
Gender: Male
Medical History: John has a history of asthma and seasonal allergies. He underwent a knee surgery in 2018 due to an accident. No known chronic conditions or heart disease. 
Medications: Currently using Ventolin inhaler for asthma as needed. No regular medications.
Allergies: Allergic to penicillin.
Emergency Contact: Jane Doe (Wife), 555-1234
Primary Care Physician: Dr. Smith, General Practitioner, ABC Health Clinic.
```

### **Instructions**:
1. **Understand the Scenario**: 
   - In this workshop, your task is to convert the medical document into a structured format that the system can process and store in a database.
   - Think about the key pieces of information you would need for a medical record (e.g., patient’s name, medical history, medications, allergies, etc.).

2. **Define the Data Structure**: 
   - Based on the given medical document, define a **Pydantic** data model that will represent the important attributes of a patient’s record.
   - Consider what fields should be included, such as **name**, **date of birth**, **gender**, **medical history**, **medications**, **allergies**, **emergency contact**, and **primary care physician**.

3. **Use the Data Model**: 
   - Your Pydantic model should reflect the data in the medical document.
   - Once you've defined the data structure, create a prompt that can be used to extract the necessary details from medical documents in the future.

4. **Key Questions**:
   - What fields would be critical for storing a patient’s medical information?
   - How can you ensure that this model can handle additional fields in the future (e.g., for future conditions, treatments, or insurance information)?

### **Deliverables**:
- Define a Pydantic data model for the personal medical information.
- Provide a structured list of fields that should be included in the model (e.g., name, DOB, medications, allergies, etc.).

In [19]:
### Example Solution:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

# Define the data structure for Personal Medical Information.
class MedicalRecord(BaseModel):
    patient_name: str = Field(description="name of the patient")
    date_of_birth: str = Field(description="date of birth of the patient")
    gender: str = Field(description="gender of the patient")
    medical_history: str = Field(description="medical history of the patient")
    medications: str = Field(description="current medications of the patient")
    allergies: str = Field(description="allergies the patient has")
    emergency_contact_name: str = Field(description="emergency contact's name")
    emergency_contact_phone: str = Field(description="emergency contact's phone number")
    primary_care_physician: str = Field(description="name of the primary care physician")

# Given query (patient medical document).
medical_document = """
Patient Name: John Doe
Date of Birth: 1985-06-15
Gender: Male
Medical History: John has a history of asthma and seasonal allergies. He underwent a knee surgery in 2018 due to an accident. No known chronic conditions or heart disease. 
Medications: Currently using Ventolin inhaler for asthma as needed. No regular medications.
Allergies: Allergic to penicillin.
Emergency Contact: Jane Doe (Wife), 555-1234
Primary Care Physician: Dr. Smith, General Practitioner, ABC Health Clinic.
"""

# Set up the JsonOutputParser and the prompt template.
parser = JsonOutputParser(pydantic_object=MedicalRecord)

template = "Extract the necessary details from the medical record.\n{format_instructions}\n{query}\n"

prompt = PromptTemplate(
    template=template,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Assuming the model is defined in your pipeline
chain = prompt | model | parser
medical_info = chain.invoke({"query": medical_document})

# Display the extracted information
print(medical_info)

{}


## Workshop: Prompt Engineering for an Insurance Assistant

### Scenario: **Insurance Inquiry Assistant** (15mins)

You are building a chatbot for an insurance company that helps users quickly get information about various insurance policies. Your task is to create prompts and integrate them into a chain for effective responses.

### Instruction:

1. **Create a Prompt Template**:
   - Imagine you are an assistant helping users understand insurance policies. Write a prompt that asks for detailed information about a specific type of insurance in a specific country.
   - Example: "You are an assistant helping a customer. Respond with a detailed explanation of the coverage for {topic} insurance in {country}. Make sure to include key benefits, eligibility, and coverage details."

2. **Connect Prompt to Chain**:
   - Link your prompt to a chain, where the assistant’s response should include information about the chosen topic and country.
   - Use `chain.invoke()` to test with different topics like "health insurance," "life insurance," or "car insurance" and countries like "Thailand," "USA," or "UK."

### **Deliverables**:
- Think about the information a customer would need when asking about an insurance policy. Include coverage details, eligibility, and other important aspects.
- Try to modify the prompt template to provide responses that are helpful, concise, and clear.

In [27]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

# Output parser for the response
output_parser = StrOutputParser()

# Write the prompt template
template = """

"""

prompt = PromptTemplate(
    template=template,
    input_variables=[]
)

# Set up the chain
chain = prompt | model | output_parser

# Invoke the chain with specific queries
response_health_thailand = chain.invoke({})
print(response_health_thailand)

Certainly! Here’s a detailed explanation of health insurance policies in Thailand, covering the key aspects you requested:

### Coverage
Health insurance policies in Thailand typically cover a range of medical services and treatments, including:

- **Inpatient Care**: Hospital stays, surgeries, and treatments requiring overnight admission.
- **Outpatient Care**: Doctor visits, specialist consultations, diagnostic tests, and minor procedures that do not require hospitalization.
- **Emergency Services**: Ambulance services and emergency room visits.
- **Preventive Care**: Vaccinations, health check-ups, and screenings.
- **Maternity Care**: Prenatal and postnatal care, childbirth, and related services.
- **Chronic Illness Management**: Ongoing treatment for chronic conditions like diabetes or hypertension.
- **Prescription Medications**: Coverage for prescribed medications, often with limits.
- **Rehabilitation Services**: Physical therapy and rehabilitation after surgery or injury.

###

## Appendix

### Corporate Company
#### good prompt 

**Context**
I'm a brand manager at Plantae, a Thai brand and products are available in Thailand only. I am developing a marketing campaign to drive the adoption of Plantae's new ready-to-drink plant-based protein product in convenience stores. For now, I'm struggling to write a good KOL brief for presenting the product. I need your help in doing this.

**Role and Backstory**
- Role: Thai expert influencer marketer
- Backstory: You have 10 years of experience working in an influencer marketing agency and directly with brands. You know exactly how to write a concise and creative KOL briefing to get a very satisfying outcome from KOLs in any post format.
  
**Key Information for Briefing KOL**
1. Key Message: "Plantae, Bottled Plant-Based Protein Drink"
2. Product: Ready-to-drink bottled plant-based protein.
  - Unique Selling Point 1: "Tasty Plant-Based Protein"
  - Unique Selling Point 2: "Available at 7-Eleven"
  - Unique Selling Point 3: "Healthy Option"
1. Sale Channel: Convenience Store
2. Mood & Tone:
  - Mood: Tasty, Healthy, Environmentally Friendly, and Convenient. An influencer post aims to acknowledge consumers that they can be healthier and more environmentally friendly by easily grabbing this drink in every convenience store.
  - Tone: Conversational and encouraging. The KOL should speak to the audience in a friendly and personal manner, explaining why people should give it a try and what are the personal
1. Color Theme: #084838, #086858, and #E8F8E8
2. Hashtag: #PlantaeOnTheGo
3. Post Format: Instagram image post.

**Instruction**
Could you create 3 KOL breifing with different unique selling points?

**Expected Outcome of Each KOL Brief**
- Key Message:
- Product:
- Unique Sellina Point: (Onlv one exact unique sellina point that I have aiven to vou).
- Mood & Tone: (Emotionally describe "Mood" and "Tone" separately)
- Color Theme: (Explain each color in simple English and also visually create the colors as a palette)
- Hashtag:
- Post Format:

#### bad prompt

Could you create 3 KOL briefs for Plantae's ready-to-drink products' Instagram posts, focusing on their unique selling points tasty plant-based protein, availability at 7-Eleven, and being a healthy option

### Insurance Industry
#### Good Prompt:

**Context**  
I'm the marketing manager for Muang Thai Life Assurance, a well-established Thai insurance company. We are launching a new life insurance product targeting young professionals aged 25-40 who are looking for affordable coverage to protect their future. I need your help to write a creative KOL brief for promoting this product on social media. 

**Role and Backstory**  
- Role: Expert influencer marketer with experience in insurance and financial services.  
- Backstory: You have worked with leading insurance brands for over 8 years, developing campaigns that connect emotionally with young audiences and clearly explain the importance of securing their future.

**Key Information for Briefing KOL**  
1. **Key Message**: "Protect Your Future with Muang Thai Life Assurance"
2. **Product**: Affordable Life Insurance for Young Professionals  
   - **Unique Selling Point 1**: "Affordable monthly premiums for young professionals"
   - **Unique Selling Point 2**: "Coverage tailored for life's unpredictable moments"
   - **Unique Selling Point 3**: "Easy application process with fast approval"
3. **Target Audience**: Young professionals aged 25-40
4. **Mood & Tone**:
   - **Mood**: Empowering, Secure, Trustworthy. The influencer should make the audience feel confident in their decision to secure their future, knowing they're protected by Muang Thai Life Assurance.
   - **Tone**: Friendly, Relatable, and Reassuring. The influencer should speak as if offering personal advice, encouraging young professionals to think ahead about their security and wellbeing.
5. **Color Theme**:  
   - **Primary Color**: #003B4D (Deep Blue) – Represents trust, security, and stability.  
   - **Secondary Color**: #FFB71B (Golden Yellow) – Evokes optimism, warmth, and financial growth.  
   - **Accent Color**: #F1F1F1 (Soft White) – Clean, approachable, and professional.
6. **Hashtag**: #SecureYourFutureWithMTL
7. **Post Format**: Instagram Carousel Post

**Expected Outcome**  
- **Key Message**: "Protect Your Future with Muang Thai Life Assurance"  
- **Product**: Affordable Life Insurance for Young Professionals  
- **Unique Selling Point**: Affordable monthly premiums for young professionals  
- **Mood & Tone**:  
   - **Mood**: Empowering, Secure, Trustworthy  
   - **Tone**: Friendly, Relatable, Reassuring  
- **Color Theme**:  
   - **#003B4D** (Deep Blue) – Trust and stability  
   - **#FFB71B** (Golden Yellow) – Optimism and growth  
   - **#F1F1F1** (Soft White) – Clean and professional  
- **Hashtag**: #SecureYourFutureWithMTL  
- **Post Format**: Instagram Carousel Post


#### Bad Prompt:

Could you create 3 KOL briefs for Muang Thai Life Assurance’s insurance products, focusing on the affordable premiums, easy application, and coverage options?